# NYC Taxi Trip OSRM Analysis Notebook

This notebook includes the analysis of the OSRM data for the NYC Taxi Trip dataset.

In [ ]:
!pip install osmnx folium networkx numpy pandas seaborn matplotlib scipy scikit-learn

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import folium
from scipy.spatial import cKDTree
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
# Helper functions
def put_text_on_barplot(ax, values):
    for i, value in enumerate(values):
        ax.text(i, value, value, ha='center')

Loading the data

In [ ]:

nyc_osrm = pd.read_csv('../datasets/osrm.csv')

In [ ]:
# load the nyc data
nyc = pd.read_csv('../datasets/nyc_with_features_basic_filtering.csv')


OSRM - Open Source Routing Machine, is a routing engine designed for use with data from the OpenStreetMap project.
To enhance our dataset, we will use the OSRM data to get the fastest routes for each trip in the NYC Taxi dataset, along with some other road conditions data.

Reference: https://project-osrm.org/

### OSRM data Example

Lets start by viewing an example of how OSRM api works.    

In [ ]:
import osmnx as ox
import networkx as nx

origin_point = (40.748817, -73.985428)
destination_point = (40.712776, -74.005974) 

G = ox.graph_from_point(origin_point, dist=10000, network_type='drive')

orig_node = ox.distance.nearest_nodes(G, origin_point[1], origin_point[0])
dest_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])

route = nx.shortest_path(G, orig_node, dest_node, weight='length')

fig, ax = ox.plot_graph_route(G, route, node_size=0)



The above example is just to show how the OSRM api works, we will use the OSRM data that specifically designed for the NYC Taxi dataset.

### NYC OSRM Dataset

The dataset contains the following columns:
- id: A unique identifier for each trip.
- distance (meters): The distance of the fastest route in meters.
- duration (seconds): The duration of the fastest route in seconds given by OSRM.
- srcCounty: The county where the trip started. 1: Manhattan, 2: Bronx, 3: Brooklyn, 4: Queens, 5: State Island.
- dstCounty: The county where the trip ended. 1: Manhattan, 2: Bronx, 3: Brooklyn, 4: Queens, 5: State Island.
Some road conditions data:
- motorway: The fraction of the fastest route that is on a motorway.
- trunk: The fraction of the fastest route that is on a trunk road.
- primary: The fraction of the fastest route that is on a primary road.
- secondary: The fraction of the fastest route that is on a secondary road.
- tertiary: The fraction of the fastest route that is on a tertiary road.
- unclassified: The fraction of the fastest route that is on an unclassified road.
- residential: The fraction of the fastest route that is on a residential road.
- nTrafficSignals: The number of traffic signals on the fastest route.
- nCrossings: The number of pedestrian crossings on the fastest route.
- nIntersections: The number of intersections on the fastest route.
- nStops: The number of stops signs on the fastest route.

For more information about the road types - https://wiki.openstreetmap.org/wiki/Key:highway#Roads

In [ ]:
nyc_osrm.head()

In [ ]:
nyc_osrm.info()

#### Exploring imperfect data

Looks like dstCounty have some missing values, lets check for missing values and duplicates in the dataset.

In [ ]:
missing_values = nyc_osrm.isnull().sum()
duplicate_values = nyc_osrm.duplicated('id').sum()
print(missing_values)
print(f"Missing values: {missing_values.sum()}")
print(f"Number of duplicates: {duplicate_values}")
print(f"Percentage of missing values: {missing_values.sum() / nyc_osrm.shape[0] * 100:.2f}%")

The missing values are in srcCounty and dstCounty, according to the dataset description, the missing values are for trips that are outside of NYC, In this case we can drop the missing values, since they are not alot, we do it later after merging the datasets.


##### Duration and Distance Imperfect Data


In [ ]:
nyc_osrm[['distance', 'duration']].describe()

There is no negative values in the distance and duration columns, but there are some values that are equal to 0.
A good way to check if the osrm distance is correct is to check if it is bigger than the haversine distance between the pickup and dropoff points.

For the duration, we can compare to the trip duration in the NYC dataset, we will do this later after merging the datasets.

##### Road Conditions Imperfect Data

First thought that all road types should sum to 1, lets check if this is the case.

In [ ]:
road_types = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential']
nyc_osrm[road_types].sum(axis=1).round(2).value_counts()

# plot the distribution of the sum of road types
plt.figure(figsize=(10, 6))
sns.histplot(nyc_osrm[road_types].sum(axis=1).round(2), bins=100, kde=True)
# set xticks to be in percentage 0% to 100%
plt.xticks(np.linspace(0, 1, num=11), labels=[f'{int(tick * 100)}%' for tick in np.linspace(0, 1, num=11)])
# set yticks to be in hundreds of thousands, for example 100k, 200k, 300k
plt.yticks(np.linspace(0, 800000, num=9), labels=[f'{int(tick / 1000)}k' for tick in np.linspace(0, 800000, num=9)])
plt.title('Distribution of the Sum of Road Types')
plt.xlabel('Sum of Road Types (%)')

The road types doesnt sum to 1, this could be for multiple reasons:
1) The road types are not complete, there might be other road types that are not included in the dataset, like sidewalk, footway, etc.
2) The road types are not accurate, the data might be noisy or not accurate.

For now, lets mark them as imprefect road types

We will use a threshold of 0.95 which is soft and not too strict, we can adjust this value later.

In [ ]:
nyc_osrm['perfect_road_types'] = nyc_osrm[road_types].sum(axis=1).round(2) > 0.95

In [ ]:
print(len(nyc_osrm[nyc_osrm['perfect_road_types']])/len(nyc_osrm)*100)

About 58% of the data is marked as perfect road types.

##### Missing Trip between osrm and nyc datasets
Lets check if some ids are missing in the OSRM dataset.

In [ ]:
missing_ids = nyc[~nyc['id'].isin(nyc_osrm['id'])]
missing_ids

In [ ]:
# drop the missing ids
nyc = nyc[nyc['id'].isin(nyc_osrm['id'])]

Now that both datasets are aligned, we can merge them together.

In [ ]:
nyc_with_osrm = nyc.merge(nyc_osrm, on='id')

### Exploring the NYC OSRM Dataset

Lets explore the merged dataset before we start the imperfection handling.

In [ ]:
nyc_with_osrm.head()

In [ ]:
nyc_with_osrm.info()

Rename some columns to prevent confusion with the original dataset.

In [ ]:
nyc_with_osrm.rename(columns={'duration': 'osrm_duration','distance': 'osrm_distance'}, inplace=True)

#### Source and Destination County Distribution

Changing the srcCounty and dstCounty to their actual names instead of the numerical values.


In [ ]:
mapping_dict = {1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'State Island'}

nyc_with_osrm['dstCountyClass'] = nyc_with_osrm['dstCounty'].map(mapping_dict)
nyc_with_osrm['srcCountyClass'] = nyc_with_osrm['srcCounty'].map(mapping_dict)

##### Map preview

We will sample some points per county and plot them on the map to get a better understanding of the data.

In [ ]:
def plot_county_map(map, data):
    county_to_color = {'Brooklyn': 'blue', 'Queens': 'green', 'State Island': 'red', 'Manhattan': 'purple', 'Bronx': 'orange'}

    for county in county_to_color:
        county_data = data[data['srcCountyClass'] == county]
        county_sample = county_data.sample(frac=0.01 if county == 'Manhattan' else 0.1) # Manhattan has a lot of data points, so we sample less
        
        for i in range(county_sample.shape[0]):
            folium.CircleMarker(
                [county_sample.iloc[i]['pickup_latitude'], county_sample.iloc[i]['pickup_longitude']], 
                radius=1, color=county_to_color[county]
            ).add_to(map)
            

nyc_map = folium.Map(location=[40.748817, -73.985428], zoom_start=10)

plot_county_map(nyc_map, nyc_with_osrm)

nyc_map


Insights: Most of the trips start in Manhattan, followed by Brooklyn and Queens. The number of trips starting in State Island and the Bronx is much lower. 

In [ ]:
def create_data_for_county_plot(data):
    src_data = data[['srcCountyClass']].copy()
    src_data['type'] = 'srcCountyClass'
    src_data.reset_index(drop=True, inplace=True)

    dst_data = data[['dstCountyClass']].copy()
    dst_data['type'] = 'dstCountyClass'
    dst_data.reset_index(drop=True, inplace=True)
    
    src_data.columns = ['CountyClass', 'type']
    dst_data.columns = ['CountyClass', 'type']
    
    return pd.concat([src_data, dst_data], ignore_index=True)

combined_data = create_data_for_county_plot(nyc_with_osrm)

plt.figure(figsize=(10, 6))
ax = sns.countplot(x='CountyClass', hue='type', data=combined_data)
plt.title('Number of Trips per County Class (srcCountyClass vs dstCountyClass)')

for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='baseline', fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')

ax.tick_params(left=False, bottom=False)

plt.tight_layout()
plt.show()


Insights: Most trips start and end in Manhattan, followed by Brooklyn and Queens. The number of trips starting and ending in State Island and the Bronx is much lower. This is expected, as Manhattan is the most densely populated and central borough in New York City.

#### Road Conditions Distribution

Because we have some imperfect data, we will first analyze a filtered dataset to get a better understanding of the road conditions.
If we will find some interesting insights, we will try to fix the imperfect data.

In [ ]:
# taking the 97.5% quantile of the trip duration, and removing osrm_duration that are less than 0
filtered_nyc_with_osrm = nyc_with_osrm[nyc_with_osrm['trip_duration'] < nyc_with_osrm['trip_duration'].quantile(0.975)]
filtered_nyc_with_osrm = filtered_nyc_with_osrm[filtered_nyc_with_osrm['osrm_duration'] > 0]

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(filtered_nyc_with_osrm[['trip_duration', 'osrm_duration', 'osrm_distance','nTrafficSignals','nCrossing','nStop','nIntersection']].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap between Trip Duration and OSRM Duration')
plt.show()

Insights: osrm duration , osrm distance and nTrafficSignals have a positive correlation with the trip duration, this could be useful for our model.

#### OSRM Duration vs Trip Duration

Lets plot the trip duration vs OSRM duration, we will use the filtered data for better visualization.

In [ ]:
plt.figure(figsize=(10, 6))
plt.hexbin(np.log(filtered_nyc_with_osrm['osrm_duration']), np.log(filtered_nyc_with_osrm['trip_duration']), gridsize=50,  mincnt=1)
log_ticks = np.linspace(np.log(filtered_nyc_with_osrm['osrm_duration']).min(), np.log(filtered_nyc_with_osrm['osrm_duration']).max(), num=10)
original_ticks = np.expm1(log_ticks)
plt.xticks(log_ticks, labels=[f'{int(tick):,}' for tick in original_ticks])
log_ticks = np.linspace(np.log(filtered_nyc_with_osrm['trip_duration']).min(), np.log(filtered_nyc_with_osrm['trip_duration']).max(), num=10)
original_ticks = np.expm1(log_ticks)
plt.yticks(log_ticks, labels=[f'{int(tick):,}' for tick in original_ticks])
plt.title('Hexbin Plot of Trip Duration vs OSRM Duration')
plt.xlabel('OSRM Duration log(seconds)')
plt.ylabel('Trip Duration log(seconds)')

cbar = plt.colorbar(label='Count')
cbar.set_ticks(cbar.get_ticks())
cbar.set_ticklabels([int(tick / 1000) for tick in cbar.get_ticks()])
cbar.set_label('Count (x1k)')

plt.show()

Insights: We can see a positive linear relationship between the trip duration and the OSRM duration.

### OSRM Distance vs Trip Duration

Lets plot the trip duration vs OSRM distance, we will use the filtered data for better visualization. 

In [ ]:
import matplotlib.pyplot as plt

# Filter the data by trip duration up to 1 hour
filtered_nyc_with_osrm = nyc_with_osrm[nyc_with_osrm['trip_duration'] < 60 * 60]
# Filter the data by OSRM distance up to 80 km (converted to meters)
filtered_nyc_with_osrm = filtered_nyc_with_osrm[filtered_nyc_with_osrm['osrm_distance'] < 80000]

plt.figure(figsize=(10, 6))
plt.hexbin(filtered_nyc_with_osrm['osrm_distance'], filtered_nyc_with_osrm['trip_duration'], gridsize=50, mincnt=1)
plt.title('Hexbin Plot of Trip Duration vs OSRM Distance')
plt.xlabel('OSRM Distance (meters)')
plt.ylabel('Trip Duration (seconds)')

cbar = plt.colorbar(label='Count')
cbar.set_ticks(cbar.get_ticks())
cbar.set_ticklabels([int(tick / 1000) for tick in cbar.get_ticks()])
cbar.set_label('Count (x1k)')

x_ticks = plt.gca().get_xticks() 
x_ticks = x_ticks[1:]
plt.xticks(x_ticks, [f'{int(tick / 1000)}k' for tick in x_ticks])  # Convert to thousands

plt.show()


Insights: There is a positive correlation between the trip duration and the OSRM distance, but the relationship is not as strong as with the OSRM duration. This could be due to traffic conditions, road conditions, or other factors that affect the trip duration, remember the imperfect osrm data that we have seen earlier.

#### Handling imperfect data
From the exploration insights looks like osrm dataset have some good information that can be used to enhance the NYC dataset, but there are some imperfect data that we need to handle.

1) Since the missing values in srcCounty and dstCounty are for trips that are outside of NYC, we can drop them, and it only 0.5% of the data.

In [ ]:
nyc_with_osrm.dropna(inplace=True)
nyc_with_osrm.reset_index(drop=True, inplace=True)
nyc_with_osrm.info()

2. Lets check the distance and duration columns, and compare them to the haversine distance and the trip duration in the NYC dataset.

In [ ]:
nyc_with_osrm['haversine_distance'].describe()

The approximated minimum block length in nyc is 80 meters, thus the minimum haversine distance should be around 80 meters, lets check for distances less than 80 meters, we can assume that most people will walk this distance.

In [ ]:
# drop rows with haversion distance less than 20 meters
nyc_with_osrm = nyc_with_osrm[nyc_with_osrm['haversine_distance'] > 80].reset_index(drop=True)

In [ ]:
nyc_with_osrm['haversine_distance'].describe()

In [ ]:
# add distance diff column
nyc_with_osrm['distance_diff'] = nyc_with_osrm['osrm_distance'] - nyc_with_osrm['haversine_distance']

In [ ]:
# check for distance smaller than the manhattan distance
nyc_with_osrm[nyc_with_osrm['osrm_distance'] < nyc_with_osrm['haversine_distance']]

In [ ]:
nyc_with_osrm[nyc_with_osrm['osrm_distance'] < nyc_with_osrm['haversine_distance']]['distance_diff'].describe()

# print the precentage of the data that have a distance smaller than the haversine distance
print(len(nyc_with_osrm[nyc_with_osrm['osrm_distance'] < nyc_with_osrm['haversine_distance']])/len(nyc_with_osrm)*100)
print(len(nyc_with_osrm[nyc_with_osrm['osrm_distance'] < nyc_with_osrm['haversine_distance']]))

Only 0.35% of the data have a distance smaller than the haversine distance.

Haversine distance is the shortest distance between two points on the surface of a sphere, we can see that there are some trips that have a distance smaller than this distance, this could be due to the imperfect data in the OSRM dataset, lets mark them as imperfect data.

In [ ]:
nyc_with_osrm['perfect_distance'] = nyc_with_osrm['osrm_distance'] >= nyc_with_osrm['haversine_distance']

Lets move on to the osrm duration, lets get the difference between the trip duration and the osrm duration.

In [ ]:
nyc_with_osrm['duration_ratio'] = nyc_with_osrm['trip_duration'] / nyc_with_osrm['osrm_duration']

In [ ]:
nyc_with_osrm['duration_diff'] = nyc_with_osrm['trip_duration'] - nyc_with_osrm['osrm_duration']

In [ ]:
nyc_with_osrm[['duration_ratio', 'duration_diff']].describe()

The basic filtering using haversine removed all those 0 osrm duration values, lets check the distribution of the duration ratio.

In [ ]:
duration_ratio = nyc_with_osrm['duration_ratio']
plt.figure(figsize=(10, 6))
sns.histplot(duration_ratio, bins=100, kde=True)

plt.title('Distribution of the Duration Ratio')
plt.xlabel('Duration Ratio')
plt.ylabel('Number of Trips')


Its hard to understand the distribution, Most of the data is around 1, lets check by splitting the data into two parts, one with duration ratio less than 1 and the other with duration ratio greater than 1.

In [ ]:
nyc_with_osrm[nyc_with_osrm['duration_ratio'] < 1][['id', 'duration_ratio', 'trip_duration', 'osrm_duration','duration_diff']].describe()

In [ ]:

# Filter the data for duration_ratio < 1
filtered_data = nyc_with_osrm[nyc_with_osrm['duration_ratio'] < 1]

plt.figure(figsize=(10, 6))

plt.scatter(filtered_data['haversine_distance'], filtered_data['duration_ratio'], alpha=0.5, label='Data Points')

plt.xlabel('Haversine Distance')
plt.ylabel('Duration Ratio')
plt.title('Haversine Distance vs Duration Ratio for Duration Ratio < 1')
plt.legend()
plt.grid(True)

plt.show()


The scatter plot shows that it looks like a polynom or a curve which we can use to remove outliers.

In [ ]:
print(len(nyc_with_osrm[nyc_with_osrm['duration_ratio'] < 0.835]))
print(len(nyc_with_osrm[nyc_with_osrm['duration_ratio'] < 0.835])/len(nyc_with_osrm)*100)
nyc_with_osrm[nyc_with_osrm['duration_ratio'] < 0.835][['id', 'duration_ratio', 'trip_duration', 'osrm_duration','duration_diff']].describe()

Using a lower bound 0.835 is a good start to filter out the imperfect data, we can adjust this value later, in this way only 0.38% of the data is marked as imperfect data.

Now for the other part:

In [ ]:
nyc_with_osrm[(nyc_with_osrm['duration_ratio'] >= 1) & (nyc_with_osrm['duration_ratio'] != np.inf)][['id', 'duration_ratio']].describe()


The mean is reasonable, around 3, but the std is very large due to the outliers.

In [ ]:
filtered_data = nyc_with_osrm[(nyc_with_osrm['duration_ratio'] >= 1) & (nyc_with_osrm['duration_ratio'] != np.inf)]

plt.figure(figsize=(10, 6))
plt.scatter(filtered_data['haversine_distance'], filtered_data['duration_ratio'], alpha=0.5, label='Data Points')

plt.xlabel('Haversine Distance')
plt.ylabel('Duration Ratio')
plt.title('Haversine Distance vs Duration Ratio for Duration Ratio >= 1')
plt.legend()
plt.grid(True)


In [ ]:
# if we use the mean as the upper bound, how much data will be marked as imperfect?
upper_bound = nyc_with_osrm['duration_ratio'].mean()
print(len(nyc_with_osrm[nyc_with_osrm['duration_ratio'] > upper_bound]))
print(len(nyc_with_osrm[nyc_with_osrm['duration_ratio'] > upper_bound])/len(nyc_with_osrm)*100)
print(upper_bound)

In this way 22.5% of the data will be marked as imperfect data, 3.5 ratio is reasonable, we will mark those as imperfect data and try to fix them using the good data.

In [ ]:
lower_bound = 0.835

nyc_with_osrm['perfect_duration'] = (nyc_with_osrm['duration_ratio'] >= lower_bound) & (nyc_with_osrm['duration_ratio'] <= upper_bound)

Now that we marked all the imperfect data, lets try to fix them.

For all imperfect data, we will try to find the nearest neighbor in the good data. 

In [ ]:
# Step 1: Create KD-Trees for pickup and dropoff coordinates
good_data = nyc_with_osrm[nyc_with_osrm['perfect_distance'] & nyc_with_osrm['perfect_duration'] & nyc_with_osrm['perfect_road_types']].reset_index(drop=True)
bad_data = nyc_with_osrm[~nyc_with_osrm['perfect_distance'] | ~nyc_with_osrm['perfect_duration'] | ~nyc_with_osrm['perfect_road_types']].reset_index(drop=True)

print(f'Number of good samples: {good_data.shape[0]}')
print(f'Number of bad samples: {bad_data.shape[0]}')

scaler = StandardScaler()
good_coords = scaler.fit_transform(good_data[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude','haversine_distance']])
bad_coords = scaler.transform(bad_data[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude',
                                        'haversine_distance']])
# Build a k-d tree for the good samples using the combined features
tree = cKDTree(good_coords)




In [ ]:
distances, indices = tree.query(bad_coords)

In [ ]:
nyc_with_osrm.loc[~nyc_with_osrm['perfect_distance'] | ~nyc_with_osrm['perfect_duration'] | ~nyc_with_osrm['perfect_road_types'], 'nearest_neighbor_id'] = good_data.iloc[indices.flatten()]['id'].values

Adding the nearest neighbor pickup and dropoff haversine distance to the bad data.

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the haversine distance between two points
    """
    R = 6371  # Radius of the Earth in kilometers
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c * 1000  # convert to meters

In [ ]:
# for each bad sample, calculate the haversine distance between it and its nearest neighbor, add the result to new columns: haversine_to_nn_pickup_distance and haversine_to_nn_dropoff_distance on nyc_with_osrm

bad_good_haversine_distances_pickup = []
bad_good_haversine_distances_dropoff = []

for i in range(bad_data.shape[0]):
    bad_sample = bad_data.iloc[i]
    nearest_neighbor = good_data.iloc[indices[i]]
    
    bad_good_haversine_distance_pickup = haversine_distance(bad_sample['pickup_latitude'], bad_sample['pickup_longitude'], nearest_neighbor['pickup_latitude'], nearest_neighbor['pickup_longitude'])
    bad_good_haversine_distance_dropoff = haversine_distance(bad_sample['dropoff_latitude'], bad_sample['dropoff_longitude'], nearest_neighbor['dropoff_latitude'], nearest_neighbor['dropoff_longitude'])
    
    bad_good_haversine_distances_pickup.append(bad_good_haversine_distance_pickup)
    bad_good_haversine_distances_dropoff.append(bad_good_haversine_distance_dropoff)
    
nyc_with_osrm.loc[~nyc_with_osrm['perfect_distance'] | ~nyc_with_osrm['perfect_duration'] | ~nyc_with_osrm['perfect_road_types'], 'haversine_to_nn_pickup_distance'] = bad_good_haversine_distances_pickup
nyc_with_osrm.loc[~nyc_with_osrm['perfect_distance'] | ~nyc_with_osrm['perfect_duration'] | ~nyc_with_osrm['perfect_road_types'], 'haversine_to_nn_dropoff_distance'] = bad_good_haversine_distances_dropoff


In [ ]:


# Create a map centered on NYC

def plot_pairs_on_map(map, data, distance_threshold: tuple = None):
    colors = ['lightgreen', 'purple', 'white', 'darkgreen', 'darkblue', 'lightblue', 'cadetblue', 'pink', 'gray', 'lightgray', 'orange', 'darkpurple', 'black', 'beige', 'red', 'darkred', 'lightred', 'green', 'blue']
    
    for i, color in enumerate(colors):
        # Extract a bad sample
        if distance_threshold:
            bad_samples_within_range = data[(data['haversine_to_nn_pickup_distance'] >= distance_threshold[0]) & (data['haversine_to_nn_pickup_distance'] <= distance_threshold[1]) & (data['haversine_to_nn_dropoff_distance'] >= distance_threshold[0]) & (data['haversine_to_nn_dropoff_distance'] <= distance_threshold[1])]
            bad_sample = bad_samples_within_range.iloc[i]
        else:
            bad_sample = data.loc[~data['perfect_distance'] | ~data['perfect_duration'] | ~data['perfect_road_types']].iloc[i]
        nearest_neighbor_id = bad_sample['nearest_neighbor_id']
        
        # Find the nearest neighbor using the ID
        nearest_neighbor = data[data['id'] == nearest_neighbor_id].iloc[0]
        
        # Add pickup and dropoff points for the bad sample
        folium.Marker([bad_sample['pickup_latitude'], bad_sample['pickup_longitude']], icon=folium.Icon(color=color)).add_to(map)
        folium.Marker([bad_sample['dropoff_latitude'], bad_sample['dropoff_longitude']], icon=folium.Icon(color=color)).add_to(map)
        
        # Add pickup and dropoff points for the nearest neighbor
        folium.Marker([nearest_neighbor['pickup_latitude'], nearest_neighbor['pickup_longitude']], icon=folium.Icon(color=color, icon='cloud')).add_to(map)
        folium.Marker([nearest_neighbor['dropoff_latitude'], nearest_neighbor['dropoff_longitude']], icon=folium.Icon(color=color, icon='cloud')).add_to(map)
        
        # Draw polylines for the pickup and dropoff points with haversine distance in the popup
        folium.PolyLine([[bad_sample['pickup_latitude'], bad_sample['pickup_longitude']], 
                         [nearest_neighbor['pickup_latitude'], nearest_neighbor['pickup_longitude']]], 
                        color='blue', popup=f'Haversine Distance: {bad_sample["haversine_to_nn_pickup_distance"]:.2f} meters').add_to(map)
        
        folium.PolyLine([[bad_sample['dropoff_latitude'], bad_sample['dropoff_longitude']], 
                         [nearest_neighbor['dropoff_latitude'], nearest_neighbor['dropoff_longitude']]], 
                        color='blue', popup=f'Haversine Distance: {bad_sample["haversine_to_nn_dropoff_distance"]:.2f} meters').add_to(map)
        
        # Draw lines between pickup and dropoff points for both the bad sample and its nearest neighbor
        folium.PolyLine([[bad_sample['pickup_latitude'], bad_sample['pickup_longitude']], 
                         [bad_sample['dropoff_latitude'], bad_sample['dropoff_longitude']]], 
                        color='red', popup=f'Haversine Distance: {bad_sample["haversine_distance"]:.2f} meters').add_to(map)
        
        folium.PolyLine([[nearest_neighbor['pickup_latitude'], nearest_neighbor['pickup_longitude']], 
                         [nearest_neighbor['dropoff_latitude'], nearest_neighbor['dropoff_longitude']]], 
                        color='red', popup=f'Haversine Distance: {nearest_neighbor["haversine_distance"]:.2f} meters').add_to(map)

nyc_map = folium.Map(location=[40.748817, -73.985428], zoom_start=10)
plot_pairs_on_map(nyc_map, nyc_with_osrm)
# Display the map
nyc_map


In [ ]:
# describe the bad data
nyc_with_osrm.loc[~nyc_with_osrm['perfect_distance'] | ~nyc_with_osrm['perfect_duration'] | ~nyc_with_osrm['perfect_road_types'], ['haversine_to_nn_pickup_distance', 'haversine_to_nn_dropoff_distance']].describe(percentiles=[0.01, 0.05, 0.95,0.975,  0.99])

In [ ]:
nyc_with_osrm.info()

If we choose the 97.5%, the haversine distance between the bad samples and their nearest neighbors is less than 500 meters, this is a reasonable value, we will use this value to fix the imperfect data, we can assume that round types in this range should be more or less the same.

In [ ]:
columns_to_update = road_types + ['osrm_distance', 'osrm_duration']
merged_df = nyc_with_osrm.merge(
    nyc_with_osrm[['id'] + columns_to_update],
    left_on='nearest_neighbor_id',
    right_on='id',
    suffixes=('', '_nn'),
    how='left'
)

# Apply the distance conditions to filter bad samples
condition = (
    (~merged_df['perfect_distance'] | ~merged_df['perfect_duration'] | ~merged_df['perfect_road_types']) &
    (merged_df['haversine_to_nn_pickup_distance'] < 500) &
    (merged_df['haversine_to_nn_dropoff_distance'] < 500)
)

# Update the columns with the nearest neighbor's data where conditions are met
for column in columns_to_update:
    merged_df.loc[condition, column] = merged_df.loc[condition, f'{column}_nn']

# Update the perfect flags
merged_df.loc[condition, 'perfect_road_types'] = True
merged_df.loc[condition, 'perfect_distance'] = True
merged_df.loc[condition, 'perfect_duration'] = True

# Drop the unnecessary columns
nyc_with_osrm = merged_df.drop([col + '_nn' for col in columns_to_update] + ['id_nn'], axis=1)

In [ ]:
print('Percentage of perfect data after fixing the imperfect data:')
print(len(nyc_with_osrm[nyc_with_osrm['perfect_distance'] & nyc_with_osrm['perfect_duration'] & nyc_with_osrm['perfect_road_types']])/len(nyc_with_osrm)*100)

In [ ]:
# get some samples with distance between 400 and 500 meters along with good samples

nyc_map = folium.Map(location=[40.748817, -73.985428], zoom_start=10)
plot_pairs_on_map(nyc_map, nyc_with_osrm, distance_threshold=(400, 500))
nyc_map

In [ ]:
# keep only samples that are marked as perfect
nyc_with_osrm = nyc_with_osrm[nyc_with_osrm['perfect_distance'] & nyc_with_osrm['perfect_duration'] & nyc_with_osrm['perfect_road_types']]

# drop the unnecessary columns
nyc_with_osrm.drop(['nearest_neighbor_id', 'haversine_to_nn_pickup_distance', 'haversine_to_nn_dropoff_distance', 'perfect_distance', 'perfect_duration', 'perfect_road_types'], axis=1, inplace=True)

In [ ]:
nyc_with_osrm.info()

In [ ]:
# reset the index
nyc_with_osrm.reset_index(drop=True, inplace=True)
nyc_with_osrm.info()

In [ ]:
# update duration ratio and duration diff
nyc_with_osrm['duration_ratio'] = nyc_with_osrm['trip_duration'] / nyc_with_osrm['osrm_duration']
nyc_with_osrm['duration_diff'] = nyc_with_osrm['trip_duration'] - nyc_with_osrm['osrm_duration']

In [ ]:
# take data with perfect distance and duration and road types and plot duration_ratio vs haversine_distance
plt.figure(figsize=(10, 6))
plt.scatter(nyc_with_osrm['haversine_distance'], nyc_with_osrm['duration_ratio'], alpha=0.5)

plt.xlabel('Haversine Distance')
plt.ylabel('Duration Ratio')
plt.title('Haversine Distance vs Duration Ratio')
plt.grid(True)
plt.show()


In [ ]:
# plot trip_duration vs haversine_distance
plt.figure(figsize=(10, 6))
plt.scatter(nyc_with_osrm['haversine_distance'], nyc_with_osrm['trip_duration'], alpha=0.5)

plt.xlabel('Haversine Distance')
plt.ylabel('Trip Duration')
plt.title('Haversine Distance vs Trip Duration')
plt.grid(True)
plt.show()

In [ ]:
# filter samples with trip duration bigger than 20000
nyc_with_osrm = nyc_with_osrm[nyc_with_osrm['trip_duration'] < 20000]

we filtered road types that are less than 0.95, we will normalize the road types that are not 1 to be 1

In [ ]:
road_types = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential']
nyc_with_osrm[road_types] = nyc_with_osrm[road_types].div(nyc_with_osrm[road_types].sum(axis=1), axis=0)

In [ ]:
nyc_with_osrm[road_types].sum(axis=1).round(1).value_counts()

In [ ]:
nyc_with_osrm.info()

#### Re-Exploring the NYC OSRM Dataset after fixing the imperfect data

1) County Distribution

In [ ]:
nyc_map = folium.Map(location=[40.748817, -73.985428], zoom_start=10)
plot_county_map(nyc_map, nyc_with_osrm)
nyc_map

In [ ]:
combined_data = create_data_for_county_plot(nyc_with_osrm)
# Plot the combined data
plt.figure(figsize=(10, 6))
sns.countplot(x='CountyClass', hue='type', data=combined_data)
plt.title('Number of Trips per County Class (srcCountyClass vs dstCountyClass)')

# Add values on top of the bars
for p in plt.gca().patches:
    plt.gca().annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='baseline', fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')
    
plt.show()

Due to the filtering, we lost most of long island trips, it is ok since we only had around 500 trips that start or end in long island.

2) Trip duration vs OSRM Duration and OSRM Distance

In [ ]:
plt.figure(figsize=(10, 6))
plt.hexbin(np.log(nyc_with_osrm['osrm_duration']), np.log(nyc_with_osrm['trip_duration']), gridsize=50,  mincnt=1)
log_ticks = np.linspace(np.log(nyc_with_osrm['osrm_duration']).min(), np.log(nyc_with_osrm['osrm_duration']).max(), num=10)
original_ticks = np.expm1(log_ticks)
plt.xticks(log_ticks, labels=[f'{int(tick):,}' for tick in original_ticks])
# do the same for the y axis
log_ticks = np.linspace(np.log(nyc_with_osrm['trip_duration']).min(), np.log(nyc_with_osrm['trip_duration']).max(), num=10)
original_ticks = np.expm1(log_ticks)
plt.yticks(log_ticks, labels=[f'{int(tick):,}' for tick in original_ticks])
plt.title('Hexbin Plot of Trip Duration vs OSRM Duration')
plt.xlabel('OSRM Duration log(seconds)')
plt.ylabel('Trip Duration log(seconds)')

cbar = plt.colorbar(label='Count')
cbar.set_ticks(cbar.get_ticks())
cbar.set_ticklabels([int(tick / 1000) for tick in cbar.get_ticks()])
cbar.set_label('Count (x1k)')

plt.show()

In [ ]:
# filter the data by trip duration up to 1 hour
filtered_nyc_with_osrm = nyc_with_osrm[nyc_with_osrm['trip_duration'] < 60 * 60]
# filter the data by osrm distance up to 100 km
filtered_nyc_with_osrm = filtered_nyc_with_osrm[filtered_nyc_with_osrm['osrm_distance'] < 80000]

plt.figure(figsize=(10, 6))
plt.hexbin(filtered_nyc_with_osrm['osrm_distance'], filtered_nyc_with_osrm['trip_duration'], gridsize=50, mincnt=1)
plt.title('Hexbin Plot of Trip Duration vs OSRM Distance')
plt.xlabel('OSRM Distance (meters)')
plt.ylabel('Trip Duration (seconds)')
cbar = plt.colorbar(label='Count')
cbar.set_ticks(cbar.get_ticks())
cbar.set_ticklabels([int(tick / 1000) for tick in cbar.get_ticks()])
cbar.set_label('Count (x1k)')
plt.show()

#### Road Conditions Analysis

We will try to understand the road conditions data in the OSRM dataset and how it relates to the trip duration.
Since we have alot of combinations of road types, we will start by grouping them into main classes.

First step is to understand how many categories we have, we will use KMeans clustering to group the road types into clusters, and find the optimal number of clusters using the elbow method.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select the road type columns for clustering
road_types = nyc_with_osrm[['motorway', 'trunk', 'primary', 'secondary',
                  'tertiary', 'unclassified', 'residential']]

# Standardize the data
scaler = StandardScaler()
road_types_scaled = scaler.fit_transform(road_types)

In [ ]:
from sklearn.cluster import KMeans

# Determine the optimal number of clusters using the elbow method
inertia = []
for n_clusters in range(1, 11):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(road_types_scaled)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8, 6))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()


From the elbow method, it looks like the optimal number of clusters is around 7, we will use this number to group the road types into clusters.

In [ ]:

# Fit the K-Means model with the chosen number of clusters
kmeans = KMeans(n_clusters=7, random_state=42, n_init=10)
clusters = kmeans.fit_predict(road_types_scaled)

# Add the cluster labels to the original DataFrame
nyc_with_osrm['roadTypeClass'] = clusters


In [ ]:
# Plot the distribution of trip duration across clusters
plt.figure(figsize=(10, 6))
sns.boxplot(x='roadTypeClass', y='trip_duration', data=nyc_with_osrm[nyc_with_osrm['trip_duration'] < nyc['trip_duration'].quantile(0.975)])
plt.title('Trip Duration by Cluster')
plt.xlabel('Road Type Class')
plt.ylabel('Trip Duration (seconds)')
plt.show()

# Calculate the mean fraction of each road type within each cluster
road_type_means = nyc_with_osrm.groupby('roadTypeClass')[['motorway', 'trunk', 'primary', 'secondary',
                                          'tertiary', 'unclassified', 'residential']].mean().reset_index()
# for each road type print the precentage of each road type in each cluster
for idx , road_type_class in road_type_means.iterrows():
    print(f'Road Type Class {idx}')
    for road_type in road_types.columns:
        print(f'{road_type} : {road_type_class[road_type]*100:.2f}%')
    print('\n')



The reason why road0 and road5 take the longest time to travel on is because they are mostly trunk and primary roads, which are typically larger and the driving distance is longer
.

#### Distance vs Road Type

In [ ]:
# group by roadTypeClass and calculate the mean distance
road_type_distance = nyc_with_osrm.groupby('roadTypeClass')['osrm_distance'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='roadTypeClass', y='osrm_distance', data=road_type_distance)
plt.title('Mean Distance by Road Type Class')
plt.xlabel('Road Type Class')
plt.ylabel('Mean Distance (meters)')
plt.show()

In [ ]:
# plot countplot for roadTypeClass
plt.figure(figsize=(10, 6))
sns.countplot(x='roadTypeClass', data=nyc_with_osrm)
plt.title('Number of Trips per Road Type Class')
plt.xlabel('Road Type Class')
plt.ylabel('Number of Trips')
plt.show()

Indeed, road0 and road5 have the longest mean distance, which is consistent with the trip duration analysis.

In [ ]:
# drop distance_diff, duration_diff, duration_ratio
nyc_with_osrm.drop(['distance_diff', 'duration_diff', 'duration_ratio'], axis=1, inplace=True)

In [ ]:
# reset the index
nyc_with_osrm.reset_index(drop=True, inplace=True)

In [ ]:
nyc_with_osrm.info()

In [ ]:
# save the data
# nyc_with_osrm.to_csv('nyc_with_osrm.csv', index=False)